# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [9]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os
import joblib

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [10]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

experiment_name = 'Hyperdrive-HFP-Exp'

experiment=Experiment(ws, experiment_name)
experiment

quick-starts-ws-293572
aml-quickstarts-293572
westeurope
510b94ba-e453-4417-988b-fbdc37b55ca7


Name,Workspace,Report Page,Docs Page
Hyperdrive-HFP-Exp,quick-starts-ws-293572,Link to Azure Machine Learning studio,Link to Documentation


In [11]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
found = False
key = "heart_failure_clinical_records"
description_text = ""

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://github.com/onuroezcelik/nd00333_Capstone_Project_Machine_Learning_Engineer/blob/master/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

print("The registered dataset is found: ",found)
df = dataset.to_pandas_dataframe()
df.describe()

The registered dataset is found:  True
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


In [12]:
dataset.take(5).to_pandas_dataframe()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [13]:
# Choose a name for your CPU cluster
amlcompute_cluster_name = "automlcluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster.')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [26]:
# Specify a Policy
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.2)

# Specify parameter sampler
param_sampling = RandomParameterSampling(
    {
        '--C' : choice(0.1,1),
        '--max_iter': choice(10,20,50)
    }
)

if "hyperdrive_training" not in os.listdir():
    os.mkdir("./hyperdrive_training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Estimator and hyperdrive config
estimator = ScriptRunConfig(
    source_directory='.',
    script='train.py',
    compute_target=amlcompute_cluster_name,
    environment=sklearn_env    
    )

hyperdrive_run_config = HyperDriveConfig(hyperparameter_sampling=param_sampling,
                                     primary_metric_name='AUC_weighted',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=early_termination_policy,
                                     run_config=estimator,
                                     max_total_runs=47,
                                     max_concurrent_runs=4)

In [27]:
#Submit experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [28]:
import collections
# Due to the not working widgets library at Python 3.10+
if not hasattr(collections, 'Mapping'):
    import collections.abc
    collections.Mapping = collections.abc.Mapping
    collections.Sequence = collections.abc.Sequence
    collections.Iterable = collections.abc.Iterable
    collections.Callable = collections.abc.Callable

from azureml.widgets import RunDetails

RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

A Jupyter Widget

RunId: HD_aafda51f-c03d-4d23-975e-c6e22031835f
Web View: https://ml.azure.com/runs/HD_aafda51f-c03d-4d23-975e-c6e22031835f?wsid=/subscriptions/510b94ba-e453-4417-988b-fbdc37b55ca7/resourcegroups/aml-quickstarts-293572/workspaces/quick-starts-ws-293572&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2026-01-12T18:22:04.6218110Z][GENERATOR][WARNING]Space size : 6 is less than max total jobs : 47, only 6 jobs will be generated 
[2026-01-12T18:22:06.1315631Z][GENERATOR][DEBUG]Sampled 4 jobs from search space 
[2026-01-12T18:22:06.4202841Z][SCHEDULER][INFO]Scheduling job, id='HD_aafda51f-c03d-4d23-975e-c6e22031835f_0' 
[2026-01-12T18:22:06.5261185Z][SCHEDULER][INFO]Scheduling job, id='HD_aafda51f-c03d-4d23-975e-c6e22031835f_1' 
[2026-01-12T18:22:06.6503427Z][SCHEDULER][INFO]Scheduling job, id='HD_aafda51f-c03d-4d23-975e-c6e22031835f_2' 
[2026-01-12T18:22:06.6568641Z][SCHEDULER][INFO]Scheduling job, id='HD_aafda51f-c03d-4d23-975e-c6e22031835f_3' 
[2026-01-12

{'runId': 'HD_aafda51f-c03d-4d23-975e-c6e22031835f',
 'target': 'automlcluster',
 'status': 'Completed',
 'startTimeUtc': '2026-01-12T18:22:04.188299Z',
 'endTimeUtc': '2026-01-12T18:35:12.488855Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"AUC_weighted","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '7d231315-258e-4b57-a094-85b8b8c5de0a',
  'user_agent': 'python/3.10.11 (Linux-6.8.0-1030-azure-x86_64-with-glibc2.35) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.60.0',
  'best_child_run_id': 'HD_aafda51f-c03d-4d23-975e-c6e22031835f_3',
  'score': '0.7666666666666667',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_aafda51f-c03d-4d23-975e-c6e22031835f_3'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azureml-

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [30]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics() 
best_run_file_names = best_run.get_file_names()

print('Best Run Id: ', best_run.id) 
print('Best Run Metrics: ', best_run_metrics) 
print('Best Run AUC_weighted:', best_run_metrics['AUC_weighted']) 
print('Best Run File Names:', best_run_file_names) 

Best Run Id:  HD_aafda51f-c03d-4d23-975e-c6e22031835f_3
Best Run Metrics:  {'Regularization Strength:': 1.0, 'Max iterations:': 50, 'AUC_weighted': 0.7666666666666667}
Best Run AUC_weighted: 0.7666666666666667
Best Run File Names: ['logs/azureml/dataprep/0/rslex.log.2026-01-12-18', 'outputs/model.joblib', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']


In [35]:
# Save the best model
hd_model = best_run.register_model(model_name='best_hd_model', model_path='outputs/')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

